In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("discussion.ipynb")

# Discussion 7

### Due Saturday November 6th, 11:59:59PM


# Web Scraping and Regex

In [24]:
import os
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import re

In [25]:
from discussion import *

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Let's scrape some books data!
* We will be scraping some book information from http://books.toscrape.com/index.html
* Note that this website is built for scraping purposes

### Get all the book urls from single catalogue page

In [4]:
url = 'http://books.toscrape.com/catalogue/page-1.html'

In [5]:
book_request = requests.get(url)
soup = BeautifulSoup(book_request.text)
soup

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:30" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="

In [6]:
articles = soup.find_all('article', {"class": "product_pod"})
articles

[<article class="product_pod">
 <div class="image_container">
 <a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
 </div>
 <p class="star-rating Three">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>
 <h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
 <div class="product_price">
 <p class="price_color">Â£51.77</p>
 <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>
 <form>
 <button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
 </form>
 </div>
 </article>,
 <article class="product_pod">
 <div class="image_container">
 <a href="tipping-the-velvet_999/index.html"><img alt="Tipping the Velvet" class="thumbnail" src="../media/cach

In [7]:
book_links = [art.find('a').get('href') for art in articles]
book_links

['a-light-in-the-attic_1000/index.html',
 'tipping-the-velvet_999/index.html',
 'soumission_998/index.html',
 'sharp-objects_997/index.html',
 'sapiens-a-brief-history-of-humankind_996/index.html',
 'the-requiem-red_995/index.html',
 'the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'the-black-maria_991/index.html',
 'starving-hearts-triangular-trade-trilogy-1_990/index.html',
 'shakespeares-sonnets_989/index.html',
 'set-me-free_988/index.html',
 'scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html',
 'rip-it-up-and-start-again_986/index.html',
 'our-band-could-be-your-life-scenes-from-the-american-indie-underground-1981-1991_985/index.html',
 'olio_984/index.html',
 'mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html',

### Parse Book information from a book URL

In [8]:
# For the first book
book_url = 'http://books.toscrape.com/catalogue/' + book_links[0]

In [9]:
book_soup = BeautifulSoup(requests.get(book_url).text)
book_soup

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    A Light in the Attic | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now

In [10]:
name = book_soup.find('article', {'class':'product_page'}).h1.text
name

'A Light in the Attic'

In [11]:
price = book_soup.find('p', {'class':'price_color'}).text
price

'Â£51.77'

In [12]:
price = float(price[2:])
price

51.77

In [13]:
rating = book_soup.find('p', class_ = re.compile('star-rating')).get('class')[1]
rating

'Three'

**Question 1**

Get all the catalog urls of the website in the form of a list. Notice that each page of the website has a very specific pattern.

In [33]:
get_website_urls()

['http://books.toscrape.com/catalogue/-0.html',
 'http://books.toscrape.com/catalogue/-1.html',
 'http://books.toscrape.com/catalogue/-2.html',
 'http://books.toscrape.com/catalogue/-3.html',
 'http://books.toscrape.com/catalogue/-4.html',
 'http://books.toscrape.com/catalogue/-5.html',
 'http://books.toscrape.com/catalogue/-6.html',
 'http://books.toscrape.com/catalogue/-7.html',
 'http://books.toscrape.com/catalogue/-8.html',
 'http://books.toscrape.com/catalogue/-9.html',
 'http://books.toscrape.com/catalogue/-10.html',
 'http://books.toscrape.com/catalogue/-11.html',
 'http://books.toscrape.com/catalogue/-12.html',
 'http://books.toscrape.com/catalogue/-13.html',
 'http://books.toscrape.com/catalogue/-14.html',
 'http://books.toscrape.com/catalogue/-15.html',
 'http://books.toscrape.com/catalogue/-16.html',
 'http://books.toscrape.com/catalogue/-17.html',
 'http://books.toscrape.com/catalogue/-18.html',
 'http://books.toscrape.com/catalogue/-19.html',
 'http://books.toscrape.com/ca

In [41]:
grader.check("q1")

q1 results: All test cases passed!

**Question 2**

Get all the category names (`Travel`, `Mystery`..) as a list. Notice to remove all the extra whitespaces if any.

In [46]:
book_request = requests.get('http://books.toscrape.com/index.html')
soup = BeautifulSoup(book_request.text)

sidebar =soup.find_all('div', {'class':'side_categories'})[0]
result = []
for cat in sidebar.find_all('a'):
    result.append(cat.text)
result = [i.strip() for i in result]
result[1:]

['Travel',
 'Mystery',
 'Historical Fiction',
 'Sequential Art',
 'Classics',
 'Philosophy',
 'Romance',
 'Womens Fiction',
 'Fiction',
 'Childrens',
 'Religion',
 'Nonfiction',
 'Music',
 'Default',
 'Science Fiction',
 'Sports and Games',
 'Add a comment',
 'Fantasy',
 'New Adult',
 'Young Adult',
 'Science',
 'Poetry',
 'Paranormal',
 'Art',
 'Psychology',
 'Autobiography',
 'Parenting',
 'Adult Fiction',
 'Humor',
 'Horror',
 'History',
 'Food and Drink',
 'Christian Fiction',
 'Business',
 'Biography',
 'Thriller',
 'Contemporary',
 'Spirituality',
 'Academic',
 'Self Help',
 'Historical',
 'Christian',
 'Suspense',
 'Short Stories',
 'Novels',
 'Health',
 'Politics',
 'Cultural',
 'Erotica',
 'Crime']

In [47]:
# don't change this cell -- it is needed for the tests to work
cats = book_categories()

E:\Workspace\DSC 80\di\07-scraping-regex\assignment\discussion.py:51: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 51 of the file E:\Workspace\DSC 80\di\07-scraping-regex\assignment\discussion.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(book_request.text)


In [48]:
grader.check("q2")

q2 results: All test cases passed!

# Regular Expressions

## Useful links

https://pythex.org/

https://www.debuggex.com/cheatsheet/regex/python

https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf

**Question 3**: Identify duplicate words in a sentence

Given an input sentence, provide a list of words that are duplicated. If there is no duplication return an empty list

In [23]:
# Use capture group number to identify duplicates

In [59]:
duplicate_words('I went to the market market with my my my family')

['market', 'my']

In [60]:
grader.check("q3")

q3 results: All test cases passed!

**Question 4**: Extract laptop specifications
The below image provides details on column names and the extract patterns. Given a df with product description - Return df with added columns of `processor` (i3, i5), `generation` (9th Gen, 10th Gen), `storage` (512 GB SSD, 1 TB HDD), `display_in_inch` (15.6 inch, 14 inch). 

If there is no specific information present, keep a null (`NaN`) value.

<img src='imgs/laptop_specs.PNG'>

In [83]:
# Use 'pd_column.str.extract(r'pattern')' to extract the required pattern
df = pd.read_csv('data/laptop_details.csv')
df

,laptop_description
0,"Intel Core i5 Processor (9th Gen), 8 GB DDR4 R..."
1,"Intel Core i3 Processor (2nd Gen), 8 GB DDR4..."
2,"Intel Core i5 Processor ( 10th Gen), 64 bit W..."
3,"Intel Core i5 Processor (10th Gen ), 256 GB SS..."
4,"Intel Core i3 Processor, 4 GB DDR4 RAM, 64 bit..."
5,"Intel Core i3 Processor (10th Gen), 4 GB DDR4 ..."
6,"Intel Core i5 Processor (10th Gen), 8 GB DDR4 ..."
7,"Intel Core i5 Processor (10th Gen), 8 GB DDR4 ..."
8,"Intel Core i3 Processor (10th Gen), 64 bit Win..."
9,"Intel Core i5 Processor (10th Gen), 1 TB HDD, ..."


In [101]:
df['laptop_description'].loc[8]

'Intel Core i3 Processor (10th Gen), 64 bit Windows 10 Operating System, 256 GB SSD, 35.56 cm (14 inch) Display, Asus Splendid, Tru2Life, 1 Year Onsite Warranty'

In [102]:
df['processor'] = df['laptop_description'].str.extract(r'(\bi\d+\b)')
df['generation'] = df['laptop_description'].str.extract(r'(\b\d+(th|nd|rd)\b\s+\bGen\b)')[0]
df['storage'] = df['laptop_description'].str.extract(r'(\b\d+\s(GB|TB)\s(SSD|HDD)\b)')[0]
df['display_inch'] = df['laptop_description'].str.extract(r'(\b\d+.?\d+\s+inch\b)')[0]
df

,laptop_description,processor,generation,storage,display_inch
0,"Intel Core i5 Processor (9th Gen), 8 GB DDR4 R...",i5,9th Gen,512 GB SSD,15.65 inch
1,"Intel Core i3 Processor (2nd Gen), 8 GB DDR4...",i3,2nd Gen,1 TB HDD,NaN
2,"Intel Core i5 Processor ( 10th Gen), 64 bit W...",i5,10th Gen,512 GB SSD,15.6 inch
3,"Intel Core i5 Processor (10th Gen ), 256 GB SS...",i5,10th Gen,256 GB SSD,14 inch
4,"Intel Core i3 Processor, 4 GB DDR4 RAM, 64 bit...",i3,NaN,1 TB HDD,15.6 inch
5,"Intel Core i3 Processor (10th Gen), 4 GB DDR4 ...",i3,10th Gen,1 TB HDD,15.65 inch
6,"Intel Core i5 Processor (10th Gen), 8 GB DDR4 ...",i5,10th Gen,1 TB HDD,14 inch
7,"Intel Core i5 Processor (10th Gen), 8 GB DDR4 ...",i5,10th Gen,512 GB HDD,14 inch
8,"Intel Core i3 Processor (10th Gen), 64 bit Win...",i3,10th Gen,256 GB SSD,14 inch
9,"Intel Core i5 Processor (10th Gen), 1 TB HDD, ...",i5,10th Gen,1 TB HDD,14.2 inch


In [108]:
# don't change this cell -- it is needed for the tests to work
out = laptop_details(pd.read_csv('data/laptop_details.csv'))

In [109]:
grader.check("q4")

q4 results: All test cases passed!

## Congratulations! You're done!

* Submit your `.py` file to Gradescope. Note that you only need to submit the `.py` file; this notebook should not be uploaded. Make sure that all of your work is in the `.py` file and not here by running the doctests: `python -m doctest discussion.py`.

In [105]:
out

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [110]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!